In [ ]:
!pip install flask-ngrok

In google colab we use ngrok for using flask

In [ ]:
%mkdir templates -p

In [ ]:
%mkdir static -p

In [ ]:
#===============writing html file directly in colab===============
%%writefile templates/index1.html
<html>

<body>
    
<form action="{{url_for('after')}}" method='POST' enctype='multipart/form-data'>
    <input type="file" name='file1'>
<input type="submit" name="btn"> 
</form>

</body>

</html>

In [ ]:
#===============writing html file directly in colab===============
%%writefile templates/after.html
<html>

<body>
    
<img src="{{url_for('static', filename='file.jpg')}}" alt="image" height=52% width = 50%>

{{data}}

</body>

</html>


In [ ]:
from flask import Flask, render_template, request
import cv2
from keras.models import load_model
from flask_ngrok import run_with_ngrok
import numpy as np

app = Flask(__name__)
run_with_ngrok(app)

app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1

@app.route('/')
def index():
    return render_template('index1.html')

@app.route('/after', methods=['GET', 'POST'])
def after():
    img = request.files['file1']

    img.save('static/file.jpg')
    image = cv2.imread('static/file.jpg',0)
    
    image = cv2.resize(image, (48,48))
    image = image.reshape(1,48,48,1)
    model = load_model(r'/content/drive/My Drive/Kaggel Competition/Facial Expression Recog/model.h5')
    prediction = model.predict(image)

    label_map =   ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

    prediction = np.argmax(prediction)

    final_prediction = label_map[prediction]

    return render_template('after.html', data=final_prediction)

if __name__ == "__main__":
    app.run()